In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table
import plotly.express as px
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "apollo19"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)
WATER_BREEDS = [
        'Chesapeake Bay Retriever', 'Labrador Retriever Mix', 'Labrador Retriever', 'Newfoundland'
]
MOUNTAIN_BREEDS = [
    'German Shepherd', 'Alaskan Malamute', 'Bernese Mountain Dog', 'Husky', 'Siberian Husky'
]
DISASTER_BREEDS = [
    'Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'
]

def make_query(filter_type: str) -> dict:
    if filter_type == 'water':
        return {
            "animal_type": "Dog",
            "breed": {"$in": WATER_BREEDS},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    if filter_type == 'mountain':
        return {
            "animal_type": "Dog",
            "breed": {"$in": MOUNTAIN_BREEDS},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    if filter_type == 'disaster':
        return {
            "animal_type": "Dog",
            "breed": {"$in": DISASTER_BREEDS},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    return{}

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
logo_tag = []
if os.path.exists(image_filename):
    encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()
    logo_tag = [html.Img(src=f'data:image/png;base64,{encoded_image}',
                         style={'height': '60px', 'margin':'8px'})]


#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(logo_tag),
    html.Center(html.B(html.H1('CS-340 Grazioso Salvare Dashboard'))),
    html.Div("Dashboard by Deje Perkins", style={'fontStyle':'italic', 'marginBottom':'8px'}),
    html.Hr(),
    
    html.Div([
        html.Label("Rescue Type"),
        dcc.RadioItems(
        id='filter',
        options=[
            {'label': 'All', 'value': 'all'},
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain/Wilderness', 'value': 'mountain'},
            {'label': 'Disaster/Individual Tracking', 'value': 'disaster'},
        ],
        value='all',
        inline=True
        ),
    ]),
    html.Hr(),
    
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        sort_action='native',
        filter_action='native',
        row_selectable='single',
        selected_rows=[],
        style_table={'overflowX': 'auto'}
    ),
    
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 

    html.Br(),
    html.Hr(),
    
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Div(id='graph-id', className='col s12 m6'),
            html.Div(id='map-id',   className='col s12 m6')
        ]
    )
])    

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter', 'value')])
def update_dashboard(filter_type):
## FIX ME Add code to filter interactive data table with MongoDB queries
#
#        
#        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#        data=df.to_dict('records')
#       
#       
#        return (data,columns)

    q = make_query(filter_type)
    recs = db.read(q)
    dff = pd.DataFrame.from_records(recs)
    if '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True)
    return dff.to_dict('records')    

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #

    #return [
    #    dcc.Graph(            
    #        figure = px.pie(df, names='breed', title='Preferred Animals')
    #    )    
    #]
    dff = pd.DataFrame(viewData) if viewData else df
    if dff.empty or 'breed' not in dff.columns:
        return html.Div("No data to display.")
    fig = px.pie(dff, names='breed', title='Preferred Breeds in Current View')
    return [dcc.Graph(figure=fig)]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def highlight_selected_columns(selected_columns=None):
    if not selected_columns:
        return []
    return [{
        'if': { 'column_id': c },
        'background_color': '#D2F3FF'
    } for c in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if not viewData:
        return html.Div("No rows to map.")
     
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if dff.shape[1] < 15:
        return html.Div("Lat/Lon columns not present in this view")
    
    row = (index or [0])[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:29188/
